In [ ]:
import sys
sys.path.append('..')
import os
from common.np import *

In [ ]:
def preprocess(text):
    text=text.lower()   #すべて小文字に変換する
    text=text.replace('.',' .') #'.'の前にスペースを挿入する
    words=text.split(' ')   #文をスペースで区切ってリストに格納する

    word_to_id={}   #｛単語:ID｝
    id_to_word={}   #{ID:単語}

    for word in words:
        if word not in word_to_id:
            id=len(word_to_id)  #単語に順にIDを振っていく
            word_to_id[word]=id
            id_to_word[id]=word

    corpus=np.array([word_to_id[w] for w in words]) #文をコーパス（単語IDに置き換えたリスト）に変換

    return corpus,word_to_id,id_to_word 

In [ ]:
def create_co_matrix(corpus,vocab_size,window_size=1):
    corpus_size=len(corpus)
    co_matrix=np.zeros((vocab_size,vocab_size),dtype=np.int32)

    for idx,word_id in enumerate(corpus):   #idx:注目している単語のidx(文中の場所) , word_id:注目している単語のID
        for i in range(1,window_size+1):
            left_idx=idx-i  #注目している単語の左側の単語の文中の場所
            right_idx=idx+i

            if left_idx>=0:
                left_word_id=corpus[left_idx]   #左側の単語のIDを取得
                co_matrix[word_id,left_word_id]+=1  #注目している単語と左側に出現した単語の共起カウントを＋１

            if right_idx<corpus_size:
                right_word_id=corpus[right_idx]
                co_matrix[word_id,right_word_id]+=1

    return co_matrix

In [ ]:
#コサイン類似度
def cos_similarity(x,y,eps=1e-8):  #eps:イプシロン,ゼロ除算対策の小さい値
    nx=x/(np.sqrt(np.sum(x**2))+eps)  #xの正規化(L2ノルムで割る)
    ny=y/(np.sqrt(np.sum(y**2))+eps)  #yの正規化(L2ノルムで割る)
    return np.dot(nx,ny)    #ベクトルの内積を取る

In [ ]:
def most_similar(query,word_to_id,id_to_word,word_matrix,top=5):
    if query not in word_to_id: #queryがword_to_idに含まれていない場合
        print('%s is not found.'%query)
        return

    print('\n[query]'+query)
    query_id=word_to_id[query]  #queryのidを取得
    query_vec=word_matrix[query_id] #queryのベクトルを取得

    vocab_size=len(id_to_word)
    similarity=np.zeros(vocab_size)

    for i in range(vocab_size):   #すべての単語とのコサイン類似度を計算してsimilarityに格納
        similarity[i]=cos_similarity(word_matrix[i],query_vec)

    # print((-1*similarity).argsort())
    count=0
    for i in (-1*similarity).argsort(): #値が高い順にソートしたインデックスを返す
        if id_to_word[i]==query:
            continue
        print('%s: %s'%(id_to_word[i],similarity[i]))

        count+=1
        if count>=top:
            return

In [ ]:
def ppmi(C,verbose=True,eps=1e-8):
    M=np.zeros_like(C,dtype=np.float32)
    N=np.sum(C) #共起行列のすべての共起回数の合計
    S=np.sum(C,axis=0)  #各単語ごとの共起回数の合計
    total=C.shape[0]*C.shape[1] #共起行列の全要素数
    cnt=0

    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi=np.log2(C[i,j]*N/(S[i]*S[j]+eps))   #PMIを計算（）
            ppmi=max(0,pmi)
            M[i,j]=ppmi
            
            if verbose:
                cnt+=1
                if cnt%(total//100+1)==0:
                    print('%.1f%%done'%(100*cnt/total))

    return M

In [ ]:
#コーパスからコンテキストとターゲットを作成する関数
def create_contexts_target(corpus,window_size=1):
    print(type(corpus))
    target=corpus[window_size:-window_size]
    contexts=[]
    for idx in range(window_size,len(corpus)-window_size):
        cs=[]
        for t in range(-window_size,window_size+1):
            if t==0:    #tがターゲット自身の場合何もしない
                continue
            cs.append(corpus[idx+t])
        contexts.append(cs)

    return np.array(contexts),np.array(target)

In [ ]:
def convert_one_hot(corpus, vocab_size):
    '''one-hot表現への変換

    :param corpus: 単語IDのリスト（1次元もしくは2次元のNumPy配列）
    :param vocab_size: 語彙数
    :return: one-hot表現（2次元もしくは3次元のNumPy配列）
    '''
    N = corpus.shape[0]

    if corpus.ndim == 1:
        one_hot = np.zeros((N, vocab_size), dtype=np.int32)
        for idx, word_id in enumerate(corpus):
            one_hot[idx, word_id] = 1

    elif corpus.ndim == 2:
        C = corpus.shape[1]
        one_hot = np.zeros((N, C, vocab_size), dtype=np.int32)
        for idx_0, word_ids in enumerate(corpus):
            for idx_1, word_id in enumerate(word_ids):
                one_hot[idx_0, idx_1, word_id] = 1

    return one_hot

In [ ]:
def clip_grads(grads, max_norm):
    total_norm = 0
    for grad in grads:
        total_norm += np.sum(grad ** 2)
    total_norm = np.sqrt(total_norm)

    rate = max_norm / (total_norm + 1e-6)
    if rate < 1:
        for grad in grads:
            grad *= rate

In [ ]:
def to_cpu(x):
    import numpy
    if type(x) == numpy.ndarray:
        return x
    return np.asnumpy(x)

In [ ]:
def to_gpu(x):
    import cupy
    if type(x) == cupy.ndarray:
        return x
    return cupy.asarray(x)

In [ ]:
def eval_perplexity(model, corpus, batch_size=10, time_size=35):
    print('evaluating perplexity ...')
    corpus_size = len(corpus)
    total_loss, loss_cnt = 0, 0
    max_iters = (corpus_size - 1) // (batch_size * time_size)
    jump = (corpus_size - 1) // batch_size

    for iters in range(max_iters):
        xs = np.zeros((batch_size, time_size), dtype=np.int32)
        ts = np.zeros((batch_size, time_size), dtype=np.int32)
        time_offset = iters * time_size
        offsets = [time_offset + (i * jump) for i in range(batch_size)]
        for t in range(time_size):
            for i, offset in enumerate(offsets):
                xs[i, t] = corpus[(offset + t) % corpus_size]
                ts[i, t] = corpus[(offset + t + 1) % corpus_size]

        try:
            loss = model.forward(xs, ts, train_flg=False)
        except TypeError:
            loss = model.forward(xs, ts)
        total_loss += loss

        sys.stdout.write('\r%d / %d' % (iters, max_iters))
        sys.stdout.flush()

    print('')
    ppl = np.exp(total_loss / max_iters)
    return ppl

In [ ]:
def eval_seq2seq(model, question, correct, id_to_char,
                 verbos=False, is_reverse=False):
    correct = correct.flatten()
    # 頭の区切り文字
    start_id = correct[0]
    correct = correct[1:]
    guess = model.generate(question, start_id, len(correct))

    # 文字列へ変換
    question = ''.join([id_to_char[int(c)] for c in question.flatten()])
    correct = ''.join([id_to_char[int(c)] for c in correct])
    guess = ''.join([id_to_char[int(c)] for c in guess])

    if verbos:
        if is_reverse:
            question = question[::-1]

        colors = {'ok': '\033[92m', 'fail': '\033[91m', 'close': '\033[0m'}
        print('Q', question)
        print('T', correct)

        is_windows = os.name == 'nt'

        if correct == guess:
            mark = colors['ok'] + '☑' + colors['close']
            if is_windows:
                mark = 'O'
            print(mark + ' ' + guess)
        else:
            mark = colors['fail'] + '☒' + colors['close']
            if is_windows:
                mark = 'X'
            print(mark + ' ' + guess)
        print('---')

    return 1 if guess == correct else 0